<a href="https://colab.research.google.com/github/MasoudMoeini/Google-Street-View-Images-Blur-Detection/blob/main/GoogLeNet_SBDN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
from PIL import Image
from tensorflow import keras
from keras.models import Sequential, Model,load_model
from keras import layers
from keras import regularizers
from keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from keras.layers import Concatenate

In [69]:
!unzip -qq images.zip

In [70]:
!unzip -qq labels.zip

In [74]:
from numpy import *
images_folder= "images/*.jpg"
labels_folder = "labels/"
def data_preprocessing(images_folder,labels_folder):
  total_img = []
  class_label=[]
  label = []
  files = glob.glob (images_folder)
  for myFile in files[:4000]:
      image_name=os.path.split(myFile)[1]
      class_label.append(float(image_name[0]))
      image = array(Image.open(myFile))
      label_image = array(Image.open(labels_folder+image_name))
      label.append (label_image)
      total_img.append (image)
  total_img = np.array(total_img)
  label = np.array(label)
  class_label=np.array(class_label) 
  label= np.reshape(label,[label.shape[0],label.shape[1],label.shape[2],label.shape[3]])
  total_img= np.reshape(total_img,[total_img.shape[0],total_img.shape[1],total_img.shape[2],total_img.shape[3]])
  #np.save('class_labels', class_label)
  #np.save('total-images',total_img)
  #np.save('label-images',label)
  return class_label,label,total_img

In [75]:
class_labels,total_labels,total_images=data_preprocessing(images_folder,labels_folder)

In [19]:
def inception(x, filters):
    # 1x1
    path1 = Conv2D(filters=filters[0], kernel_size=(1,1), strides=1, padding='same', activation='relu')(x)

    # 1x1->3x3
    path2 = Conv2D(filters=filters[1][0], kernel_size=(1,1), strides=1, padding='same', activation='relu')(x)
    path2 = Conv2D(filters=filters[1][1], kernel_size=(3,3), strides=1, padding='same', activation='relu')(path2)
    
    # 1x1->5x5
    path3 = Conv2D(filters=filters[2][0], kernel_size=(1,1), strides=1, padding='same', activation='relu')(x)
    path3 = Conv2D(filters=filters[2][1], kernel_size=(5,5), strides=1, padding='same', activation='relu')(path3)

    # 3x3->1x1
    path4 = MaxPooling2D(pool_size=(3,3), strides=1, padding='same')(x)
    path4 = Conv2D(filters=filters[3], kernel_size=(1,1), strides=1, padding='same', activation='relu')(path4)

    return Concatenate(axis=-1)([path1,path2,path3,path4])

In [32]:
def auxiliary(x, name=None):
    layer = AveragePooling2D(pool_size=(5,5), strides=3, padding='valid')(x)
    layer = Conv2D(filters=128, kernel_size=(1,1), strides=1, padding='same', activation='relu')(layer)
    layer = Flatten()(layer)
    layer = Dense(units=256, activation='relu')(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(units=1, activation='softmax', name=name)(layer)
    return layer


In [64]:
def identifier(x,name=None):
  layer = layers.Conv2DTranspose(1024, (3, 3), strides=(1, 1), padding='same',activation="relu")(x)
  layer = layers.Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same',activation="relu")(layer)
  layer = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same',activation="relu")(layer)
  layer = layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same',activation="relu")(layer)
  layer = layers.Conv2DTranspose(3, (3, 3), strides=(2, 2), padding='same',activation="relu")(layer)
  layer = layers.Conv2DTranspose(3, (3, 3), strides=(2, 2), padding='same',activation="relu")(layer)
  layer = layers.Conv2DTranspose(3, (3, 3), strides=(1, 1), padding='same',activation="sigmoid")(layer)
  layer = layers.Conv2DTranspose(3, (3, 3), strides=(1, 1), padding='same',activation="sigmoid",name=name)(layer)
  return layer


In [65]:
def googlenet():
    layer_in = Input(shape=(224, 224, 3))
    
    # stage-1
    layer = Conv2D(filters=64, kernel_size=(7,7), strides=2, padding='same', activation='relu')(layer_in)
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)
    layer = BatchNormalization()(layer)

    # stage-2
    layer = Conv2D(filters=64, kernel_size=(1,1), strides=1, padding='same', activation='relu')(layer)
    layer = Conv2D(filters=192, kernel_size=(3,3), strides=1, padding='same', activation='relu')(layer)
    layer = BatchNormalization()(layer)
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)

    # stage-3
    layer = inception(layer, [ 64,  (96,128), (16,32), 32]) #3a
    layer = inception(layer, [128, (128,192), (32,96), 64]) #3b
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)
    
    # stage-4
    layer = inception(layer, [192,  (96,208),  (16,48),  64]) #4a
    aux1  = auxiliary(layer, name='aux1')
    layer = inception(layer, [160, (112,224),  (24,64),  64]) #4b
    layer = inception(layer, [128, (128,256),  (24,64),  64]) #4c
    layer = inception(layer, [112, (144,288),  (32,64),  64]) #4d
    aux2  = auxiliary(layer, name='aux2')
    layer = inception(layer, [256, (160,320), (32,128), 128]) #4e
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)
    
    # stage-5
    layer = inception(layer, [256, (160,320), (32,128), 128]) #5a
    layer = inception(layer, [384, (192,384), (48,128), 128]) #5b
    ident  = identifier(layer, name='ident')
    layer = AveragePooling2D(pool_size=(7,7), strides=1, padding='valid')(layer)
    
    # stage-6
    layer = Flatten()(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(units=256, activation='linear')(layer)
    main = Dense(units=1, activation='sigmoid', name='main')(layer)
    
    model = Model(inputs=layer_in, outputs=[main, aux1, aux2,ident])
    
    return model

In [68]:
model = googlenet()
#keras.utils.plot_model(model, show_shapes=True)

In [76]:
loss1="binary_crossentropy"
loss2=tf.keras.losses.MeanSquaredError()
model.compile(keras.optimizers.Adam(learning_rate=1e-3),loss = [loss1,loss1,loss1,loss2],loss_weights={'main': 1.0, 'aux1': 0.3, 'aux2': 0.3,'ident':1.0},metrics=["accuracy"])

In [77]:
class_label=class_labels[:3000]
x_train = total_images[:3000].astype(np.float32)/ 255.0
y_train=total_labels[:3000].astype(np.float32)/ 255.0
y_train = ( y_train,class_label)

In [78]:
history = model.fit(x_train,[y_train[1],y_train[1],y_train[1],y_train[0]], epochs=5,batch_size=20, validation_split=0.2,shuffle=True)
#model.save('googleNet_model.h5')

Epoch 1/5
120/120 [==============================] - 818s 7s/step - loss: 1.2565 - main_loss: 0.7196 - aux1_loss: 0.6916 - aux2_loss: 0.6979 - ident_loss: 0.1201 - main_accuracy: 0.4954 - aux1_accuracy: 0.5000 - aux2_accuracy: 0.5000 - ident_accuracy: 0.0043 - val_loss: 1.1985 - val_main_loss: 0.6963 - val_aux1_loss: 0.6937 - val_aux2_loss: 0.6941 - val_ident_loss: 0.0859 - val_main_accuracy: 0.4783 - val_aux1_accuracy: 0.4783 - val_aux2_accuracy: 0.4783 - val_ident_accuracy: 9.3670e-04
Epoch 2/5
  3/120 [..............................] - ETA: 11:52 - loss: 1.0610 - main_loss: 0.6288 - aux1_loss: 0.5748 - aux2_loss: 0.6298 - ident_loss: 0.0708 - main_accuracy: 0.6000 - aux1_accuracy: 0.5333 - aux2_accuracy: 0.5333 - ident_accuracy: 0.0011

KeyboardInterrupt: ignored